In [2]:
import torch
import sys
import torch
import os.path as osp
import tqdm
import sys
import numpy as np

sys.path.append('/eos/home-g/gkrzmanc/mlpf_data/')
from src.dataset.dataset import SimpleIterDataset
from src.utils.utils import to_filelist
from torch.utils.data import DataLoader
import dgl
from sklearn.decomposition import PCA


class Args:
    def __init__(self):
        self.data_train = ["/eos/user/g/gkrzmanc/ClicDet_v1_25_07_2023_full.root"]
        self.data_val = ['/eos/user/g/gkrzmanc/ClicDet_v1_25_07_2023_full.root']
        #self.data_train = files_train
        self.data_config = '/eos/home-g/gkrzmanc/mlpf_data/config_files/config_2_newlinks.yaml'
        self.extra_selection = None
        self.train_val_split = 0.8
        self.data_fraction = 1
        self.file_fraction = 1
        self.fetch_by_files = False
        self.fetch_step = 0.01
        self.steps_per_epoch = None
        self.in_memory = False
        self.local_rank = None
        self.copy_inputs = False
        self.no_remake_weights = False
        self.batch_size = 10
        self.num_workers = 0
        self.demo = False
        self.laplace = False
        self.diffs = False
        self.class_edges = False
        self.gpus = ""


args = Args()
train_range = (0, args.train_val_split)
train_file_dict, train_files = to_filelist(args, 'train')

train_data = SimpleIterDataset(train_file_dict, args.data_config, for_training=True,
                               extra_selection=args.extra_selection,
                               remake_weights=True,
                               load_range_and_fraction=(train_range, args.data_fraction),
                               file_fraction=args.file_fraction,
                               fetch_by_files=args.fetch_by_files,
                               fetch_step=args.fetch_step,
                               infinity_mode=False,
                               in_memory=args.in_memory,
                               async_load=False,
                               name='train', dataset_cap=2)

iterator = iter(train_data)


!!! Dataset_cap flag set, disabling shuffling
=== Restarting DataIter train, seed=None ===
!!! Dataset_cap flag set, disabling shuffling
=== Restarting DataIter train, seed=None ===


In [3]:
g, y = next(iterator)

In [4]:
from src.dataset.functions_graph import graph_batch_func

train_loader = DataLoader(train_data, batch_size=2, drop_last=True, pin_memory=True,
                          num_workers=min(args.num_workers, int(len(train_files) * args.file_fraction)),
                          collate_fn=graph_batch_func,
                          persistent_workers=args.num_workers > 0 and args.steps_per_epoch is not None)

In [5]:
iterator = iter(train_data)
g, y = next(iterator)

!!! Dataset_cap flag set, disabling shuffling
=== Restarting DataIter train, seed=None ===


In [6]:
y.shape

torch.Size([2, 7])

In [7]:
len(dgl.unbatch(train_loader[0]))

TypeError: 'DataLoader' object is not subscriptable

In [8]:
lst = list(train_data)

!!! Dataset_cap flag set, disabling shuffling
=== Restarting DataIter train, seed=None ===


In [9]:
len(lst)

2

In [10]:
lst[0][0]

Graph(num_nodes=805, num_edges=5635,
      ndata_schemes={'h': Scheme(shape=(9,), dtype=torch.float32), 'pos_hits': Scheme(shape=(3,), dtype=torch.float32), 'pos_hits_norm': Scheme(shape=(3,), dtype=torch.float32), 'hit_type': Scheme(shape=(4,), dtype=torch.int64), 'p_hits': Scheme(shape=(1,), dtype=torch.float32), 'e_hits': Scheme(shape=(1,), dtype=torch.float32), 'particle_number': Scheme(shape=(), dtype=torch.float32), 'particle_number_nomap': Scheme(shape=(), dtype=torch.float32)}
      edata_schemes={})

In [11]:
g

Graph(num_nodes=805, num_edges=5635,
      ndata_schemes={'h': Scheme(shape=(9,), dtype=torch.float32), 'pos_hits': Scheme(shape=(3,), dtype=torch.float32), 'pos_hits_norm': Scheme(shape=(3,), dtype=torch.float32), 'hit_type': Scheme(shape=(4,), dtype=torch.int64), 'p_hits': Scheme(shape=(1,), dtype=torch.float32), 'e_hits': Scheme(shape=(1,), dtype=torch.float32), 'particle_number': Scheme(shape=(), dtype=torch.float32), 'particle_number_nomap': Scheme(shape=(), dtype=torch.float32)}
      edata_schemes={})

In [12]:
b = next(iter(train_loader))

!!! Dataset_cap flag set, disabling shuffling
=== Restarting DataIter train, seed=None ===


In [25]:
(dgl.unbatch(b[0])[1].ndata["h"] - lst[1][0].ndata["h"]).sum()

tensor(0.)

In [27]:
dgl.unbatch(b[0])[1].ndata["particle_number"]

tensor([3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [28]:
lst[1][0].ndata["particle_number"]

tensor([3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])